In [0]:
# Install necessarry libraries and build the Object Deetection API

!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib

!git clone https://github.com/tensorflow/models.git

%cd /content/models/research

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

!python object_detection/builders/model_builder_test.py

In [0]:
# Connect Google-Drive

from google.colab import drive
drive.mount('/content/gdrive')
!ln -s "/content/gdrive/My Drive" "/content/mydrive"

In [0]:
# List the content of the local computer folder 

!ls -la "/content/mydrive/faster_rcnn_2"

In [0]:
# Tensorboard

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [0]:
# Tensorboard

LOG_DIR = '/content/mydrive/faster_rcnn_2/CP_SSD'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
print(LOG_DIR)

In [0]:
# Tensorboard

get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
# Tensorboard

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# Training for Faster - RCNN and SSD model legacy version

!python object_detection/legacy/train.py \
--train_dir=/content/mydrive/faster_rcnn_2/CP_SSD \
--pipeline_config_path=/content/mydrive/faster_rcnn_2/ssd_mobilenet_v2_quantized_300x300_coco.config \
--logtostderr

In [0]:
# Training for Faster - RCNN and SSD model new version

!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path=/content/mydrive/faster_rcnn_2/faster_rcnn_resnet50_coco.config \
    --model_dir=/content/mydrive/faster_rcnn_2/CP \
    --alsologtostderr \
    --num_train_steps=25000 \
    --num_eval_steps=1000

In [0]:
# Save inference graph (Faster - RCNN)

!python object_detection/export_inference_graph.py \
--input_type=image_tensor \
--pipeline_config_path=/content/mydrive/faster_rcnn_2/faster_rcnn_resnet50_coco.config \
--trained_checkpoint_prefix=/content/mydrive/faster_rcnn_2/CP/model.ckpt-25000 \
--output_directory=/content/mydrive/faster_rcnn_2/IG_25000

In [0]:
# Tensorflow LITE inference graph (SSD)

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=/content/mydrive/faster_rcnn_2/ssd_mobilenet_v2_quantized_300x300_coco.config \
--trained_checkpoint_prefix=/content/mydrive/faster_rcnn_2/CP_SSD/model.ckpt-23121 \
--output_directory=/content/mydrive/faster_rcnn_2/IG_SSD \
--add_postprocessing_op=true

In [0]:
# Tensorflow LITE convert pb to TFLite format (SSD)

tflite_convert \
--graph_def_file=/content/mydrive/faster_rcnn_2/IG_SSD/tflite_graph.pb \
--output_file=/content/mydrive/faster_rcnn_2/LITE_flatbuffer/detect.tflite \
--output_format=TFLITE \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_dev_values=127 \
--change_concat_input_ranges=false \
--allow_custom_ops

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


In [0]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

In [0]:
# Test on video

MODEL_NAME = '/content/mydrive/faster_rcnn_2/IG'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = '/content/mydrive/faster_rcnn_2/labelmap.pbtxt'

In [0]:
# Test on video

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [0]:
# Test on video

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
# Test on video

def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [0]:
# Test on video

import cv2

cap = cv2.VideoCapture('/content/mydrive/darknet/vili.mp4')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter('/content/mydrive/faster_rcnn_2/videos/kesz_4.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))

try:
    with detection_graph.as_default():
        with tf.Session() as sess:
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
                ]:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)

                while True:
                    ret, image_np = cap.read()
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=8)
                    out.write(image_np)

except Exception as e:
    cap.release()
    out.release()